In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_5217: Some(http://gw02.itversity.com:4048)

In [2]:
%showtypes on

Types will be printed.


In [3]:
// val spark2 = spark
// import spark2.implicits._
// import org.apache.spark.sql.SQLImplicits

Name: Syntax Error.
Message: 
StackTrace: 

In [4]:
import org.apache.spark.sql.DataFrame
import util.Random

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler

In [5]:
val dataWithoutHeader = spark
                        .read
                        .option("inferSchema",true)
                        .option("header",false)
                        .csv("hdfs:///user/kranthidr/dataSets/KDDCup1999DataSet")

Waiting for a Spark session to start...

dataWithoutHeader = [_c0: int, _c1: string ... 40 more fields]


[_c0: int, _c1: string ... 40 more fields]

In [6]:
println(dataWithoutHeader.getClass())

class org.apache.spark.sql.Dataset


In [7]:
dataWithoutHeader.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: d

In [8]:
dataWithoutHeader.take(3)

0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,normal.


In [9]:
val dataDF = dataWithoutHeader.toDF(
        "duration", "protocol_type", "service", "flag",
        "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
        "hot", "num_failed_logins", "logged_in", "num_compromised",
        "root_shell", "su_attempted", "num_root", "num_file_creations",
        "num_shells", "num_access_files", "num_outbound_cmds",
        "is_host_login", "is_guest_login", "count", "srv_count",
        "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
        "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
        "dst_host_count", "dst_host_srv_count",
        "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
        "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
        "dst_host_serror_rate", "dst_host_srv_serror_rate",
        "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
        "label").cache()


dataDF = [duration: int, protocol_type: string ... 40 more fields]


[duration: int, protocol_type: string ... 40 more fields]

In [10]:
dataDF.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [11]:
val numericOnly = dataDF.drop("protocol_type", "service", "flag")

numericOnly = [duration: int, src_bytes: int ... 37 more fields]


[duration: int, src_bytes: int ... 37 more fields]

In [12]:
println(numericOnly.getClass())
println(dataDF.getClass())

class org.apache.spark.sql.Dataset
class org.apache.spark.sql.Dataset


In [13]:
def clusterScore1(data: DataFrame, k: Int) = {
    
    val assembler = new VectorAssembler().setInputCols(data.columns.filter(_!="label")).setOutputCol("featureVector")
    
    val kmeans = new KMeans().
        setSeed(Random.nextLong()).
        setK(k).
        setMaxIter(40).
        setTol(1.0e-5).
        setPredictionCol("cluster").
        setFeaturesCol("featureVector")
    
    val pipeline = new Pipeline().setStages(Array(assembler, kmeans))
    
    val pipelineModel = pipeline.fit(data)
    
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.computeCost(pipelineModel.transform(data))/ data.count()
}

clusterScore1: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [14]:
val k_kScore1 = (30 to 210 by 30).map(k => (k, clusterScore1(numericOnly, k))) 

k_kScore1 = Vector((30,2.289764131601541E7), (60,8470305.30670904), (90,7095383.880835814), (120,4857860.611231016), (150,5993846.780990699), (180,5215423.540799868), (210,6989822.2398211965))


Vector((30,2.289764131601541E7), (60,8470305.30670904), (90,7095383.880835814), (120,4857860.611231016), (150,5993846.780990699), (180,5215423.540799868), (210,6989822.2398211965))

In [15]:
def clusterScore2(data: DataFrame, k: Int) = {
    
    val assembler = new VectorAssembler().setInputCols(data.columns.filter(_!="label")).setOutputCol("featureVector")
    
    val scalar = new StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").
        setWithStd(true).
        setWithMean(false)
    
     val kmeans = new KMeans().
        setSeed(Random.nextLong()).
        setK(k).
        setMaxIter(40).
        setTol(1.0e-5).
        setFeaturesCol("scaledFeatureVector").
        setPredictionCol("cluster")
   
    val pipeline = new Pipeline().setStages(Array(assembler, scalar, kmeans))
    
    val pipelineModel = pipeline.fit(data)
    
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.computeCost(pipelineModel.transform(data))/ data.count()
}

clusterScore2: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [16]:
val k_kScore2 = (30 to 210 by 30).map(k => (k, clusterScore2(numericOnly, k))) 

k_kScore2 = Vector((30,5.030221895764377), (60,1.1585834080429747), (90,0.7305037798805409), (120,0.4999779246027878), (150,0.4005762879937056), (180,0.33266622855714906), (210,0.2796240282743122))


Vector((30,5.030221895764377), (60,1.1585834080429747), (90,0.7305037798805409), (120,0.4999779246027878), (150,0.4005762879937056), (180,0.33266622855714906), (210,0.2796240282743122))

In [17]:
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

def oneHotPipeline(inputCol: String):(Pipeline, String) = {
    val indexer = new StringIndexer().setInputCol(inputCol).setOutputCol(inputCol + "_indexed")
    val encoder = new OneHotEncoder().setInputCol(inputCol + "_indexed").setOutputCol(inputCol + "_vec")
    val pipeline = new Pipeline().setStages(Array(indexer, encoder))
    (pipeline, inputCol +"_vec")
}

oneHotPipeline: (inputCol: String)(org.apache.spark.ml.Pipeline, String)


In [18]:
def clusterScore3(data: DataFrame, k: Int) = {
    
    val (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    val (flagEncoder, flagVecCol) = oneHotPipeline("flag")

    // Original columns, without label / string columns, but with new vector encoded cols
    val assembleCols = Set(data.columns: _*) --
      Seq("label", "protocol_type", "service", "flag")--
      Seq("protocol_type_indexed", "service_indexed", "flag_indexed") ++
      Seq(protoTypeVecCol, serviceVecCol, flagVecCol)
    
    val assembler = new VectorAssembler().
      setInputCols(assembleCols.toArray).
      setOutputCol("featureVector")
    
    val scalar = new StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").
        setWithStd(true).
        setWithMean(false)
    
     val kmeans = new KMeans().
        setSeed(Random.nextLong()).
        setK(k).
        setMaxIter(40).
        setTol(1.0e-5).
        setPredictionCol("cluster").
        setFeaturesCol("scaledFeatureVector")
   
    val pipeline = new Pipeline().setStages(
      Array(protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scalar, kmeans))
    
    val pipelineModel = pipeline.fit(data)
    
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.computeCost(pipelineModel.transform(data))/ data.count()
}

clusterScore3: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [19]:
val k_kScore3 = (30 to 210 by 30).map(k => (k, clusterScore3(dataDF, k))) 

k_kScore3 = Vector((30,67.9055123176595), (60,38.632426873025395), (90,16.138219633617414), (120,3.204927274610112), (150,2.313322225784033), (180,1.6983896850381774), (210,1.2994952266802888))


Vector((30,67.9055123176595), (60,38.632426873025395), (90,16.138219633617414), (120,3.204927274610112), (150,2.313322225784033), (180,1.6983896850381774), (210,1.2994952266802888))

In [20]:
//%AddDeps co.theasi plotly_2.11 0.2.0 --transitive

Name: Syntax Error.
Message: 
StackTrace: 

In [21]:
%addjar http://central.maven.org/maven2/co/theasi/plotly_2.11/0.2.0/plotly_2.11-0.2.0.jar

Starting download from http://central.maven.org/maven2/co/theasi/plotly_2.11/0.2.0/plotly_2.11-0.2.0.jar
Finished download of plotly_2.11-0.2.0.jar


In [22]:
import co.theasi.plotly._

In [23]:
val xs =  k_kScore1.map(_._1)
val ys =  k_kScore1.map(_._2)

Name: Compile Error
Message: <console>:46: error: not found: value k_kScore1
       val xs =  k_kScore1.map(_._1)
                 ^
<console>:47: error: not found: value k_kScore1
       val ys =  k_kScore1.map(_._2)
                 ^

StackTrace: 

In [24]:
val plot = Plot().withScatter(xs, ys)

draw(plot, "clustering_elbow_ch05_traffic_anamoly1")

Name: Compile Error
Message: <console>:46: error: not found: value xs
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                     ^
<console>:46: error: not found: value ys
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                         ^

StackTrace: 

In [25]:
val xs =  k_kScore2.map(_._1)
val ys =  k_kScore2.map(_._2)

Name: Compile Error
Message: <console>:46: error: not found: value k_kScore2
       val xs =  k_kScore2.map(_._1)
                 ^
<console>:47: error: not found: value k_kScore2
       val ys =  k_kScore2.map(_._2)
                 ^

StackTrace: 

In [26]:
val plot = Plot().withScatter(xs, ys)

draw(plot, "clustering_elbow_ch05_traffic_anamoly2")

Name: Compile Error
Message: <console>:46: error: not found: value xs
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                     ^
<console>:46: error: not found: value ys
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                         ^

StackTrace: 

In [27]:
val xs =  k_kScore3.map(_._1)
val ys =  k_kScore3.map(_._2)

Name: Compile Error
Message: <console>:46: error: not found: value k_kScore3
       val xs =  k_kScore3.map(_._1)
                 ^
<console>:47: error: not found: value k_kScore3
       val ys =  k_kScore3.map(_._2)
                 ^

StackTrace: 

In [28]:
val plot = Plot().withScatter(xs, ys)

draw(plot, "clustering_elbow_ch05_traffic_anamoly3")

Name: Compile Error
Message: <console>:46: error: not found: value xs
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                     ^
<console>:46: error: not found: value ys
Error occurred in an application involving default arguments.
       val plot = Plot().withScatter(xs, ys)
                                         ^

StackTrace: 

In [29]:
sca

Name: Compile Error
Message: <console>:47: error: not found: value sca
       sca
       ^

StackTrace: 